In [32]:
import pandas as pd
from db import call_db_json
sql = """ 
    SELECT
        o.order_number,
        o.product_family,
        o.product_model,
        o.item_description,
        o.order_quantity,
        o.product_length,
        o.reference_number,
        l.first_time_out,
        l.last_time_out,
        facility
    FROM
        manufacturing_orders o
    JOIN (
        SELECT
            order_number,
            MIN(time_out) AS first_time_out,            
            MAX(time_out) AS last_time_out,
            string_agg(l.facility_id, ' ') as facility
        FROM
            manufacturing_order_logs l
        GROUP BY
            order_number
    ) l ON o.order_number = l.order_number
    WHERE
        l.first_time_out >= '2022-01-01' AND l.last_time_out <= '2023-06-30'
        AND o.product_family IS NOT NULL
        AND o.order_release_code = 5
        AND o.order_status = '55'        
        AND o.item_description LIKE '%RAIL%'
    """
df = pd.DataFrame(call_db_json(sql))

In [33]:
df['days'] = df['last_time_out'] - df['first_time_out']
df['total_days'] = df['days'] / pd.to_timedelta(1, unit='D')
df['total_days'] = df['total_days'].apply(lambda x: "{:.1f}".format(x))
df['first_time_out'] = df['first_time_out'].astype(str)
df['last_time_out'] = df['last_time_out'].astype(str)
df = df.drop(columns=['days'])

In [30]:
# def get_gy(facility):
#     if "B0010" in facility:
#         return "GY1"
#     elif "C0010" in facility:
#         return "GY3"
#     elif "D0010" in facility:
#         return "GY4"
#     else:
#         return "etc"
# df['gy'] = df['facility'].apply(get_gy)


In [34]:
df
# df = df.drop(columns=['days'])
# df = df.rename(columns={'total_days_td':'total_days'})
df.to_excel('test.xlsx')